<a href="https://colab.research.google.com/github/TheSamurai1/Python-stuff/blob/main/MachineLearnign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
matches = pd.read_csv("matches.csv", index_col = 0)
matches["team"].value_counts() #counts the number of matches per team
matches["round"].value_counts() #numbers of matches played during each "match week"

#machinelearning alg can only work with numeric values..

matches["date"] = pd.to_datetime(matches["date"])
#changing the date column which was a string to a datetime value == numeric

matches["venue_code"] = matches["venue"].astype("category").cat.codes

#wants to make the venue col into a predictor, so changing it to a numeric value

matches["opp_code"] = matches["opponent"].astype("category").cat.codes

matches["hour"] = matches["time"].str.replace(":.+", "", regex = True).astype("int")
#ok so basically continuing to change and replace some stuff from specific
#columns, we are just doing some basic regex, same as r regex syntax for time

matches["day_code"] = matches["date"].dt.dayofweek #gets the day of week property to that col
#just makes sunday = sun , tuesday =tues, etc.

matches["target"] = (matches["result"] == "W").astype("int")

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

train = matches[matches["date"] < '2022-01-01']
#making sure the model is less than 2022 since we don't want any of that data
predictors = ["venue_code", "opp_code", "hour", "day_code"]
#we alraedy set these values up before hand
rf.fit(train[predictors], train["target"])
#we are using these as predictors to train the machine
preds = rf.predict(test[predictors])

from sklearn.metrics import accuracy_score

acc = accuracy_score(test["target"], preds)

combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns = combined["prediction"])
#predicts if we are wrong and right, we need to predict our wins

from sklearn.metrics import precision_score
#what percentage of the time did the team actually win
precision_score(test["target"], preds)

grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")

def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, colsed = 'left').mean()
  #if we dont pass closed left it uses knowledge of the future which
  #is what we dont want
  group[new_cols] = rolling_stats
  return group


cols = ["gf","ga", "sh","sot","dist","fk","pk","pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

matches_rolling = matches.groupby("team").apply(lambda x:rolling_averages(x, cols, new_cols))
#grouping by each team in the data, apply the function to compute the rolling averages for each team
#so it should do it for every match

matches_rolling = matches_rolling.droplevel("team")

matches_rolling.index= range(matches_rolling.shape[0])

def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

combined, precision = make_predicitons(matches_rolling, predictors + new_cols)

combined = combined.merge(matches_rolling[["date", "team"]], left_index = True, right_index = True)

class MissingDict(dict):
  __missing__ = lambda self, key: key

maps_values = (
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
)

mapping = MissingDict(**map_values)

combined["new_team"] = combined["team"].map(mapping)

merged = combined.merge(combined, left_on = ["date", "new_team"], right_on["date", "opponent"])

merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()
